In [1]:
import shutil
import os, sys
import subprocess
import pandas as pd
import torchaudio
import numpy as np
from indextts.infer import IndexTTS

import torch

In [2]:
#[可配置]

roota = "./pre_deal/enhancement/mix_dnor"

root = roota + "/"

output_root = "./new_data/compare/1"
os.makedirs(root, exist_ok=True)

task = pd.read_csv("./modify_text_not_only_chinese.csv") #文本修正

# os.makedirs("data_deal/data_deal_normalized/", exist_ok=True) 原
# task = pd.read_csv("aigc_speech_generation_tasks/aigc_speech_generation_tasks.csv") 原

In [3]:
#被截断的音频ID： [6, 16, 26, 42, 44, 57, 70, 79, 82, 86, 98, 104, 105, 117, 126, 128, 177, 180, 188, 193]
# 最后成功对应的num_beams： {1: 200, 2: 40, 3: 30, 4: 70, 5: 40, 6: 30, 8: 70, 9: 30, 10: 60, 11: 50, 12: 40, 13: 40, 14: 40, 15: 60, 16: 120, 17: 20, 18: 50, 19: 30, 20: 60, 21: 25, 22: 30, 23: 60, 24: 40, 25: 50, 26: 20, 27: 25, 28: 30, 29: 30, 30: 30, 31: 60, 32: 40, 33: 20, 34: 200, 35: 50, 36: 50, 37: 50, 38: 25, 39: 40, 40: 25, 41: 70, 42: 25, 43: 30, 44: 40, 45: 50, 46: 60, 47: 30, 48: 40, 49: 40, 50: 25, 51: 70, 52: 20, 53: 30, 54: 30, 55: 50, 56: 25, 57: 50, 58: 50, 59: 50, 60: 20, 61: 40, 62: 50, 63: 50, 64: 40, 65: 70, 66: 70, 67: 20, 68: 25, 69: 50, 70: 40, 71: 50, 72: 40, 73: 70, 74: 100, 75: 40, 76: 25, 77: 20, 78: 20, 79: 50, 80: 20, 81: 30, 82: 30, 83: 130, 84: 25, 85: 60, 86: 120, 87: 50, 88: 40, 89: 40, 90: 30, 91: 130, 92: 60, 93: 25, 94: 20, 95: 40, 96: 25, 97: 25, 98: 60, 99: 30, 100: 60, 101: 25, 102: 60, 103: 60, 104: 20, 105: 30, 106: 20, 107: 40, 108: 40, 109: 50, 110: 60, 111: 50, 112: 60, 113: 30, 114: 40, 115: 30, 116: 20, 117: 20, 118: 40, 119: 20, 120: 60, 121: 40, 122: 150, 123: 50, 124: 40, 125: 190, 126: 60, 127: 30, 128: 40, 129: 20, 130: 40, 131: 50, 132: 30, 133: 110, 134: 20, 135: 110, 136: 40, 137: 25, 138: 30, 139: 30, 140: 30, 141: 30, 142: 30, 143: 130, 144: 50, 145: 120, 146: 50, 147: 20, 148: 60, 149: 25, 150: 40, 151: 50, 152: 160, 153: 40, 154: 50, 155: 20, 156: 25, 157: 50, 158: 30, 159: 110, 160: 40, 161: 20, 162: 20, 163: 60, 164: 30, 165: 40, 166: 25, 167: 30, 168: 50, 169: 30, 170: 60, 171: 50, 172: 30, 173: 20, 174: 40, 175: 110, 176: 40, 177: 60, 178: 30, 179: 40, 180: 40, 181: 50, 182: 50, 183: 40, 184: 50, 185: 30, 186: 40, 187: 25, 188: 25, 189: 20, 190: 25, 191: 60, 192: 40, 193: 110, 194: 200, 195: 60, 196: 20, 197: 110, 198: 25, 199: 20, 200: 30}

In [5]:
#[可配置]

MAX_LEN_SECONDS = 50  
# 存储需要截断的音频ID
truncated_ids = []
fail_ids = []
# short_audio_ids = []  # 存储小于5秒的音频ID
short_audio_ids = [1, 2, 4, 6, 8, 14, 15, 16, 20, 21, 22, 23, 35, 37, 40, 41, 43, 52, 53, 61, 64, 65, 66, 67, 68, 71, 72, 74, 81, 84, 85, 89, 90, 91, 92, 94, 95, 99, 102, 109, 111, 112, 113, 114, 116, 119, 120, 123, 125, 130, 131, 134, 138, 139, 142, 143, 144, 147, 148, 149, 150, 151, 152, 155, 156, 161, 162, 164, 165, 166, 168, 169, 170, 172, 173, 175, 182, 183, 184, 186, 191, 194, 195, 197, 198, 200]
last_successful_beams = {1: 200, 2: 40, 3: 30, 4: 70, 5: 40, 6: 30, 8: 70, 9: 30, 10: 60, 11: 50, 12: 40, 13: 40, 14: 40, 15: 60, 16: 120, 17: 20, 18: 50, 19: 30, 20: 60, 21: 25, 22: 30, 23: 60, 24: 40, 25: 50, 26: 20, 27: 25, 28: 30, 29: 30, 30: 30, 31: 60, 32: 40, 33: 20, 34: 200, 35: 50, 36: 50, 37: 50, 38: 25, 39: 40, 40: 25, 41: 70, 42: 25, 43: 30, 44: 40, 45: 50, 46: 60, 47: 30, 48: 40, 49: 40, 50: 25, 51: 70, 52: 20, 53: 30, 54: 30, 55: 50, 56: 25, 57: 50, 58: 50, 59: 50, 60: 20, 61: 40, 62: 50, 63: 50, 64: 40, 65: 70, 66: 70, 67: 20, 68: 25, 69: 50, 70: 40, 71: 50, 72: 40, 73: 70, 74: 100, 75: 40, 76: 25, 77: 20, 78: 20, 79: 50, 80: 20, 81: 30, 82: 30, 83: 130, 84: 25, 85: 60, 86: 120, 87: 50, 88: 40, 89: 40, 90: 30, 91: 130, 92: 60, 93: 25, 94: 20, 95: 40, 96: 25, 97: 25, 98: 60, 99: 30, 100: 60, 101: 25, 102: 60, 103: 60, 104: 20, 105: 30, 106: 20, 107: 40, 108: 40, 109: 50, 110: 60, 111: 50, 112: 60, 113: 30, 114: 40, 115: 30, 116: 20, 117: 20, 118: 40, 119: 20, 120: 60, 121: 40, 122: 150, 123: 50, 124: 40, 125: 190, 126: 60, 127: 30, 128: 40, 129: 20, 130: 40, 131: 50, 132: 30, 133: 110, 134: 20, 135: 110, 136: 40, 137: 25, 138: 30, 139: 30, 140: 30, 141: 30, 142: 30, 143: 130, 144: 50, 145: 120, 146: 50, 147: 20, 148: 60, 149: 25, 150: 40, 151: 50, 152: 160, 153: 40, 154: 50, 155: 20, 156: 25, 157: 50, 158: 30, 159: 110, 160: 40, 161: 20, 162: 20, 163: 60, 164: 30, 165: 40, 166: 25, 167: 30, 168: 50, 169: 30, 170: 60, 171: 50, 172: 30, 173: 20, 174: 40, 175: 110, 176: 40, 177: 60, 178: 30, 179: 40, 180: 40, 181: 50, 182: 50, 183: 40, 184: 50, 185: 30, 186: 40, 187: 25, 188: 25, 189: 20, 190: 25, 191: 60, 192: 40, 193: 110, 194: 200, 195: 60, 196: 20, 197: 110, 198: 25, 199: 20, 200: 30}
# 基础参数配置
# base_kwargs = {
#     "do_sample": True,
#     "top_p": 0.8,
#     "top_k": 30,
#     "temperature": 1.0,
#     "length_penalty": 0.0,
#     "num_beams": 3,
#     "repetition_penalty": 10.0,
#     "output_scores": True,
# }
base_kwargs = {
    "do_sample": True,
    "top_p": 0.8,
    "top_k": 30,
    "temperature": 1.0,
    "length_penalty": 0.0,
    "num_beams": 3,
    "repetition_penalty": 10.0,
    "output_scores": True,
}

# 短音频的特殊参数配置（让克隆更像）
short_audio_kwargs = {
    "top_p": 0.8,      # 更高的top_p增加多样性但保持质量
    "top_k": 30,       # 更低的top_k减少随机性
    "temperature": 1.0, # 更低的temperature减少随机性，使输出更确定
}

tts = IndexTTS(model_dir="./index-tts/checkpoints", cfg_path="./index-tts/checkpoints/config.yaml", use_cuda_kernel=True)

# 首先识别短音频（小于5秒）
# print("正在识别短音频（<5秒）...")
# for row in task.iterrows():
#     utt_id = row[1].utt
#     voice = root + str(row[1].reference_speech)
    
#     try:
#         # 加载音频并检查时长
#         waveform, sample_rate = torchaudio.load(voice)
#         duration = waveform.shape[1] / sample_rate
        
#         if duration < 5.0:
#             short_audio_ids.append(utt_id)
#             print(f"识别为短音频: {utt_id} ({duration:.2f}s)")
#     except Exception as e:
#         print(f"无法读取音频 {utt_id}: {str(e)}")

# print(f"识别到 {len(short_audio_ids)} 个短音频")

# 使用之前记录的last_successful_beams为每个音频设置最优num_beams
print("\n开始使用最优num_beams处理音频...")
for row in task.iterrows():
    utt_id = row[1].utt
    
    # 如果这个音频ID在last_successful_beams中有记录，使用最优值
    if utt_id in last_successful_beams:
        optimal_num_beams = last_successful_beams[utt_id]
    else:
        # 如果没有记录，使用默认值
        optimal_num_beams = 20
        print(f"警告: 音频 {utt_id} 没有最优num_beams记录，使用默认值20")
    
    # 准备参数
    kwargs = base_kwargs.copy()
    kwargs["num_beams"] = optimal_num_beams
    
    # 如果是短音频，使用特殊参数
    if utt_id in short_audio_ids:
        kwargs.update(short_audio_kwargs)
        print(f"处理短音频 {utt_id}，使用特殊参数，num_beams={optimal_num_beams}")
    else:
        print(f"处理音频 {utt_id}，num_beams={optimal_num_beams}")
    
    torch.cuda.empty_cache()
    voice = root + str(row[1].reference_speech)
    text = str(row[1].text)
    output_path = output_root + "_output/synthesized_speech_" + str(utt_id) + ".wav"

    # # 音频截断逻辑（如果需要）
    # waveform, sample_rate = torchaudio.load(voice)
    # duration = waveform.shape[1] / sample_rate
    # if duration > MAX_LEN_SECONDS:
    #     print(f"警告：音频 {voice} 过长 ({duration:.1f}s > {MAX_LEN_SECONDS}s)，将截断")
    #     max_samples = int(MAX_LEN_SECONDS * sample_rate)
    #     waveform = waveform[:, :max_samples]
    #     truncated_ids.append(utt_id)
    #     truncated_path = f"result/temp_truncated_{utt_id}.wav"
    #     torchaudio.save(truncated_path, waveform, sample_rate)
    #     voice = truncated_path

    try:
        tts.infer(voice, text, output_path, **kwargs)
        print(f"处理成功: {utt_id}.wav (num_beams={optimal_num_beams})")
    except Exception as e:
        fail_ids.append(utt_id)
        print(f"处理失败 {utt_id}: {str(e)}")
    finally:
        # 清理临时文件（如果有）
        pass

# 打印最终统计结果
print("\n=== 最终处理结果 ===")
print(f"成功处理的音频数量: {len([id for id in last_successful_beams.keys() if id not in fail_ids])}")
print(f"失败的音频数量: {len(fail_ids)}")
print(f"短音频数量: {len(short_audio_ids)}")
print(f"使用截断的音频数量: {len(truncated_ids)}")

print("\n短音频ID列表:", short_audio_ids)
print("失败的音频ID:", fail_ids)

# # 保存配置信息到文件（可选）
# import json
# config_summary = {
#     "short_audio_ids": short_audio_ids,
#     "fail_ids": fail_ids,
#     "truncated_ids": truncated_ids,
#     "optimal_beams": last_successful_beams
# }

# with open("processing_summary.json", "w", encoding="utf-8") as f:
#     json.dump(config_summary, f, ensure_ascii=False, indent=2)

# print("\n处理配置已保存到 processing_summary.json")

>> GPT weights restored from: ./index-tts/checkpoints/gpt.pth
[2025-08-24 16:10:18,846] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/root/miniconda3/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::ostream::tellp()@GLIBCXX_3.4'
/root/minicond

[2025-08-24 16:10:20,942] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False
[2025-08-24 16:10:20,956] [INFO] [logging.py:107:log_dist] [Rank -1] DeepSpeed info: version=0.17.4, git-hash=unknown, git-branch=unknown
[2025-08-24 16:10:20,958] [WARNING] [config_utils.py:70:_process_deprecated_field] Config parameter mp_size is deprecated use tensor_parallel.tp_size instead
[2025-08-24 16:10:20,958] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False
[2025-08-24 16:10:20,959] [INFO] [logging.py:107:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1
>> Preload custom CUDA kernel for BigVGAN <module 'anti_alias_activation_cuda' from '/root/autodl-tmp/voice_model/index-tts/indextts/BigVGAN/alias_free_activation/cuda/build/anti_alias_activation_cuda.so'>
Removing weight norm...
>> bigvgan weights restored from: ./index-tts/checkpoints/bigvgan_generator

2025-08-24 16:10:24,113 WETEXT INFO found existing fst: /root/autodl-tmp/voice_model/index-tts/indextts/utils/tagger_cache/zh_tn_tagger.fst
2025-08-24 16:10:24,114 WETEXT INFO                     /root/autodl-tmp/voice_model/index-tts/indextts/utils/tagger_cache/zh_tn_verbalizer.fst
2025-08-24 16:10:24,114 WETEXT INFO skip building fst for zh_normalizer ...
2025-08-24 16:10:24,536 WETEXT INFO found existing fst: /root/miniconda3/lib/python3.12/site-packages/tn/en_tn_tagger.fst
2025-08-24 16:10:24,537 WETEXT INFO                     /root/miniconda3/lib/python3.12/site-packages/tn/en_tn_verbalizer.fst
2025-08-24 16:10:24,538 WETEXT INFO skip building fst for en_normalizer ...


>> TextNormalizer loaded
>> bpe model loaded from: ./index-tts/checkpoints/bpe.model

开始使用最优num_beams处理音频...
处理短音频 1，使用特殊参数，num_beams=200
>> start inference...
>> Reference audio length: 3.04 seconds
>> gpt_gen_time: 1.04 seconds
>> gpt_forward_time: 0.03 seconds
>> bigvgan_time: 0.36 seconds
>> Total inference time: 1.45 seconds
>> Generated audio length: 0.60 seconds
>> RTF: 2.4226
>> wav file saved to: ./new_data/compare/1_output/synthesized_speech_1.wav
处理成功: 1.wav (num_beams=200)
处理短音频 2，使用特殊参数，num_beams=40
>> start inference...
>> Reference audio length: 3.13 seconds
>> gpt_gen_time: 3.26 seconds
>> gpt_forward_time: 0.02 seconds
>> bigvgan_time: 0.33 seconds
>> Total inference time: 3.65 seconds
>> Generated audio length: 6.49 seconds
>> RTF: 0.5636
>> wav file saved to: ./new_data/compare/1_output/synthesized_speech_2.wav
处理成功: 2.wav (num_beams=40)
处理音频 3，num_beams=30
>> start inference...
>> Reference audio length: 6.57 seconds
>> gpt_gen_time: 4.07 seconds
>> gpt_forward_time

/root/autodl-tmp/voice_model/index-tts/indextts/utils/front.py:392: RuntimeWarning: The tokens length of sentence exceeds limit: 120, Tokens in sentence: ['▁', '您', '▁', '需', '▁', '要', '▁', '上', '▁', '传', '▁', '医', '▁', '保', '▁', '电', '▁', '子', '▁', '凭', '▁', '证', '▁', '或', '▁', '社', '▁', '会', '▁', '保', '▁', '障', '▁', '卡', '▁', '或', '▁', '身', '▁', '份', '▁', '证', '▁', '或', '▁', '户', '▁', '口', '▁', '簿', '▁', '或', '▁', '护', '▁', '照', '▁', '或', '▁', '港', '▁', '澳', '▁', '居', '▁', '民', '▁', '来', '▁', '往', '▁', '内', '▁', '地', '▁', '通', '▁', '行', '▁', '证', '▁', '或', '▁', '港', '▁', '澳', '▁', '台', '▁', '居', '▁', '民', '▁', '居', '▁', '住', '▁', '证', '▁', '或', '▁', '中', '▁', '华', '▁', '人', '▁', '民', '▁', '共', '▁', '和', '▁', '国', '▁', '外', '▁', '国', '▁', '人', '▁'].Maybe unexpected behavior
  warnings.warn(


>> Reference audio length: 15.09 seconds
>> gpt_gen_time: 6.45 seconds
>> gpt_forward_time: 0.05 seconds
>> bigvgan_time: 0.50 seconds
>> Total inference time: 7.04 seconds
>> Generated audio length: 13.82 seconds
>> RTF: 0.5096
>> wav file saved to: ./new_data/compare/1_output/synthesized_speech_42.wav
处理成功: 42.wav (num_beams=25)
处理短音频 43，使用特殊参数，num_beams=30
>> start inference...
>> Reference audio length: 2.56 seconds
>> gpt_gen_time: 3.71 seconds
>> gpt_forward_time: 0.02 seconds
>> bigvgan_time: 0.30 seconds
>> Total inference time: 4.05 seconds
>> Generated audio length: 8.23 seconds
>> RTF: 0.4924
>> wav file saved to: ./new_data/compare/1_output/synthesized_speech_43.wav
处理成功: 43.wav (num_beams=30)
处理音频 44，num_beams=40
>> start inference...
>> Reference audio length: 17.57 seconds
>> gpt_gen_time: 2.73 seconds
>> gpt_forward_time: 0.02 seconds
>> bigvgan_time: 0.26 seconds
>> Total inference time: 3.04 seconds
>> Generated audio length: 5.59 seconds
>> RTF: 0.5445
>> wav file sa